# Data Cleaning



In this notebook you will find the used pipeline to clean the datasets that are at our disposition, such as treating missing values, removing inconsistent data and transform all this datasets into ones that follow a number of normalization principles that manage them to be easier to manipulate in the Analysis Notebook.

## Libraries

In [69]:
import numpy as np
import pandas as pd

In [70]:
# in case of using colab
from google.colab import drive
drive.mount('/content/drive')

current_directory = "/content/drive/MyDrive/Proyectos/Suicidios_Análisis/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## General view of the datasets

### Loading the Data

Suicides registered since 2017.

In [71]:
df_suicides = pd.read_csv(current_directory+"Data/SAT-SS-BU_2017-2022.csv", sep=",")

In [72]:
df_suicides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21719 entries, 0 to 21718
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   id_hecho                     21719 non-null  int64 
 1   federal                      21719 non-null  object
 2   provincia_id                 21719 non-null  int64 
 3   provincia_nombre             21719 non-null  object
 4   departamento_id              21719 non-null  int64 
 5   departamento_nombre          21719 non-null  object
 6   localidad_id                 21719 non-null  object
 7   localidad_nombre             21719 non-null  object
 8   anio                         21719 non-null  int64 
 9   mes                          21719 non-null  int64 
 10  fecha_hecho                  21719 non-null  object
 11  hora_hecho                   21719 non-null  object
 12  tipo_lugar                   21719 non-null  object
 13  tipo_lugar_ampliado          21

In [73]:
df_suicides.head()

,id_hecho,federal,provincia_id,provincia_nombre,departamento_id,departamento_nombre,localidad_id,localidad_nombre,anio,mes,...,tipo_lugar_ampliado,tipo_lugar_otro,modalidad,modalidad_ampliado,modalidad_otro,motivo_origen_registro,motivo_origen_registro_otro,suicida_sexo,suicida_tr_edad,suicida_identidad_genero
0,8529,No,42,La Pampa,42002,Norte (General Pico),105030,General Pico,2017,1,...,Vía pública,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar
1,8530,No,42,La Pampa,42003,Sur (General Acha),70030,Guatraché,2017,1,...,Domicilio particular,NaN,Arma de fuego,Arma de fuego,NaN,Intervención policial,NaN,Masculino,55-59,Sin determinar
2,8569,No,78,Santa Cruz,78021,Güer Aike,60,28 de Noviembre,2017,1,...,Vía pública,NaN,Ahorcamiento,Ahorcamiento,NaN,Denuncia particular,NaN,Masculino,15-19,Sin determinar
3,8705,No,58,Neuquén,58035,Confluencia,70,Neuquén,2017,1,...,Hotel/Motel/Hospedaje temporario,Hotel/Motel/Hospedaje temporario,Arma de fuego,Arma de fuego,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar
4,8706,No,58,Neuquén,58035,Confluencia,100,Plottier,2017,1,...,Domicilio particular,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar


In [74]:
df_suicides["suicida_identidad_genero"].value_counts()

Sin determinar          14890
Varón                    5309
Mujer                    1515
Mujer trans/travesti        4
Varón trans                 1
Name: suicida_identidad_genero, dtype: int64

In [75]:
df_suicides["modalidad_ampliado"].value_counts()

Ahorcamiento                                          14651
Arma de fuego                                          3454
Sin determinar                                         1381
Se arroja al vacío                                      755
Envenenamiento                                          576
Se arroja a las vías de FF.CC.                          279
Arma blanca /  elemento cortante                        265
Sumersión en piscina / mar / río                        171
Se incinera                                             133
Se arroja bajo rodado (camion/automóvil/colectivo)       36
Otra modalidad                                           18
Name: modalidad_ampliado, dtype: int64


Crime statistics since 2000.

In [76]:
# this one uses ";" even if it is called ".csv"
df_crimes = pd.read_csv(current_directory+"Data/snic-provincias.csv", sep=";", error_bad_lines=False)

<ipython-input-76-867ebba2552b>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_crimes = pd.read_csv(current_directory+"Data/snic-provincias.csv", sep=";", error_bad_lines=False)


In [77]:
df_crimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   provincia_id               21344 non-null  int64  
 1   provincia_nombre           21344 non-null  object 
 2   anio                       21344 non-null  int64  
 3   codigo_delito_snic_id      21344 non-null  object 
 4   codigo_delito_snic_nombre  21344 non-null  object 
 5   cantidad_hechos            21344 non-null  int64  
 6   cantidad_victimas          7608 non-null   float64
 7   cantidad_victimas_masc     7608 non-null   float64
 8   cantidad_victimas_fem      7608 non-null   float64
 9   cantidad_victimas_sd       7608 non-null   float64
 10  tasa_hechos                21344 non-null  object 
 11  tasa_victimas              7608 non-null   object 
 12  tasa_victimas_masc         7608 non-null   object 
 13  tasa_victimas_fem          7608 non-null   obj

In [78]:
df_crimes.head()

,provincia_id,provincia_nombre,anio,codigo_delito_snic_id,codigo_delito_snic_nombre,cantidad_hechos,cantidad_victimas,cantidad_victimas_masc,cantidad_victimas_fem,cantidad_victimas_sd,tasa_hechos,tasa_victimas,tasa_victimas_masc,tasa_victimas_fem
0,2,Ciudad Autónoma de Buenos Aires,2000,1,Homicidios dolosos,149,158.0,121.0,37.0,0.0,"4,9750013","5,2755051","8,8257008","2,2783601"
1,2,Ciudad Autónoma de Buenos Aires,2000,2,Homicidios dolosos en grado de tentativa,54,58.0,40.0,18.0,0.0,"1,8030207","1,9365778","2,9175868","1,1083914"
2,2,Ciudad Autónoma de Buenos Aires,2000,3,Muertes en accidentes viales,121,126.0,97.0,29.0,0.0,"4,040102","4,2070484","7,0751481","1,7857418"
3,2,Ciudad Autónoma de Buenos Aires,2000,4,Homicidios culposos por otros hechos,35,36.0,20.0,13.0,3.0,"1,1686245","1,2020137","1,4587934",",80050492"
4,2,Ciudad Autónoma de Buenos Aires,2000,5,Lesiones dolosas,13154,13732.0,8105.0,5485.0,142.0,"439,20248","458,50146","591,17603","337,7515"


Wage Mean Income by Department and CLAE2 since 2014.

In [79]:
df_mean_income = pd.read_csv(current_directory+"Data/w_mean_depto_tot_emp_clae2.csv", sep=",")

In [80]:
df_mean_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3597481 entries, 0 to 3597480
Data columns (total 5 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   fecha                      object 
 1   codigo_departamento_indec  float64
 2   id_provincia_indec         float64
 3   clae2                      int64  
 4   w_mean                     int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 137.2+ MB


In [81]:
df_mean_income.head(10)

,fecha,codigo_departamento_indec,id_provincia_indec,clae2,w_mean
0,2014-01-01,2000.0,2.0,1,13085
1,2014-01-01,2000.0,2.0,2,10081
2,2014-01-01,2000.0,2.0,3,17837
3,2014-01-01,2000.0,2.0,5,57688
4,2014-01-01,2000.0,2.0,6,62211
5,2014-01-01,2000.0,2.0,7,50522
6,2014-01-01,2000.0,2.0,8,50411
7,2014-01-01,2000.0,2.0,9,41279
8,2014-01-01,2000.0,2.0,10,16659
9,2014-01-01,2000.0,2.0,11,23518


In [82]:
df_mean_income["fecha"].unique()

array(['2014-01-01', '2014-02-01', '2014-03-01', '2014-04-01',
       '2014-05-01', '2014-06-01', '2014-07-01', '2014-08-01',
       '2014-09-01', '2014-10-01', '2014-11-01', '2014-12-01',
       '2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01',
       '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01',
       '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01',
       '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
       '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01',
       '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01',
       '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01',
       '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01',
       '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01',
       '2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
       '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
       '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
       '2019-01-01', '2019-02-01', '2019-03-01', '2019-

Provinces and Departments of Argentina by INDEC's standard.

In [83]:
df_locations = pd.read_csv(current_directory+"Data/diccionario_cod_depto.csv", sep=",")

In [84]:
df_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   codigo_departamento_indec  510 non-null    int64 
 1   nombre_departamento_indec  510 non-null    object
 2   id_provincia_indec         510 non-null    int64 
 3   nombre_provincia_indec     510 non-null    object
dtypes: int64(2), object(2)
memory usage: 16.1+ KB


In [85]:
df_locations.head()

,codigo_departamento_indec,nombre_departamento_indec,id_provincia_indec,nombre_provincia_indec
0,2000,CABA,2,CABA
1,6007,Adolfo Alsina,6,Buenos Aires
2,6014,Adolfo Gonzales Chaves,6,Buenos Aires
3,6021,Alberti,6,Buenos Aires
4,6028,Almirante Brown,6,Buenos Aires


CLAE2 dictionary

In [86]:
df_clae2 = pd.read_csv(current_directory+"Data/diccionario_clae2.csv", sep=",")

In [87]:
df_clae2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   clae2       86 non-null     int64 
 1   clae2_desc  86 non-null     object
 2   letra       85 non-null     object
 3   letra_desc  86 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.8+ KB


In [88]:
df_clae2.head()

,clae2,clae2_desc,letra,letra_desc
0,1,"Agricultura, ganadería, caza y servicios relac...",A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y ..."
1,2,Silvicultura y explotación forestal,A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y ..."
2,3,Pesca y acuicultura,A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y ..."
3,5,Extracción de carbón y lignito,B,EXPLOTACION DE MINAS Y CANTERAS
4,6,Extracción de petróleo crudo y gas natural,B,EXPLOTACION DE MINAS Y CANTERAS


### Strategy

We will create the corresponding datasets that map the Relational Model that follows this ERD (Entity Relation Diagram):

![Suicide Analysis ERD](https://drive.google.com/uc?id=1islpEu6Dd5pdxKj9CztsnAgzJIQvN42F)



We will not preserve all the columns for each dataset. The most important will remain, the ones that are shared with other tables (like the province or the department) will form part of other table and the ones that we not consider significant will be dropped out.

Once that is done, we will get deeper into each of the generated tables so we make sure that the data is consistent and useful for our purpose.

Then, we can finally start with the Analysis part.

## Mapping to the relational model

### INDEC Locations

We have two independent entites: the provinces (PROVINCIA) and the CLAEs (CLAE2). With the INDEC locations dataset we will be able to get the provinces and, also, the departments that are related to them.

In [89]:
df_locations

,codigo_departamento_indec,nombre_departamento_indec,id_provincia_indec,nombre_provincia_indec
0,2000,CABA,2,CABA
1,6007,Adolfo Alsina,6,Buenos Aires
2,6014,Adolfo Gonzales Chaves,6,Buenos Aires
3,6021,Alberti,6,Buenos Aires
4,6028,Almirante Brown,6,Buenos Aires
...,...,...,...,...
505,90105,Tafí Viejo,90,Tucuman
506,90112,Trancas,90,Tucuman
507,90119,Yerba Buena,90,Tucuman
508,94007,Río Grande/Tolhuin,94,Tierra Del Fuego


In [90]:
provinces = df_locations[["id_provincia_indec", "nombre_provincia_indec"]].copy()

In [91]:
provinces.drop_duplicates(inplace=True)

In [92]:
provinces.reset_index(inplace=True)

In [93]:
provinces.drop(axis=1, labels=["index"], inplace=True)

In [94]:
provinces

,id_provincia_indec,nombre_provincia_indec
0,2,CABA
1,6,Buenos Aires
2,10,Catamarca
3,14,Cordoba
4,18,Corrientes
5,22,Chaco
6,26,Chubut
7,30,Entre Rios
8,34,Formosa
9,38,Jujuy


We go the 24 provinces (counting CABA) from Argentina with their INDEC ids and names.

In [95]:
provinces.columns = [["id_indec", "nombre_indec"]]

In [96]:
provinces

,id_indec,nombre_indec
0,2,CABA
1,6,Buenos Aires
2,10,Catamarca
3,14,Cordoba
4,18,Corrientes
5,22,Chaco
6,26,Chubut
7,30,Entre Rios
8,34,Formosa
9,38,Jujuy


Now we can proceed with the departments

In [97]:
departments = df_locations[["codigo_departamento_indec", "nombre_departamento_indec", "id_provincia_indec"]].copy()

In [98]:
departments.drop_duplicates(inplace=True)

In [99]:
departments.reset_index(inplace=True)

In [100]:
departments.drop(axis=1, labels=["index"], inplace=True)

In [101]:
departments

,codigo_departamento_indec,nombre_departamento_indec,id_provincia_indec
0,2000,CABA,2
1,6007,Adolfo Alsina,6
2,6014,Adolfo Gonzales Chaves,6
3,6021,Alberti,6
4,6028,Almirante Brown,6
...,...,...,...
505,90105,Tafí Viejo,90
506,90112,Trancas,90
507,90119,Yerba Buena,90
508,94007,Río Grande/Tolhuin,94


In [102]:
departments.columns = ["id_indec", "nombre_indec", "id_provincia_indec"]

In [103]:
departments

,id_indec,nombre_indec,id_provincia_indec
0,2000,CABA,2
1,6007,Adolfo Alsina,6
2,6014,Adolfo Gonzales Chaves,6
3,6021,Alberti,6
4,6028,Almirante Brown,6
...,...,...,...
505,90105,Tafí Viejo,90
506,90112,Trancas,90
507,90119,Yerba Buena,90
508,94007,Río Grande/Tolhuin,94


This should be enough for this dataset by now.

Since there is a relation 1 to N between the provinces and the departments, these last ones will retain their corresponding province id.

In [104]:
del df_locations

### Crimes

From this one we will get the DELITO entity model. Most of the columns will remain, we only have to make sure that their province id corresponds with the INDEC's one.

In [105]:
df_crimes

,provincia_id,provincia_nombre,anio,codigo_delito_snic_id,codigo_delito_snic_nombre,cantidad_hechos,cantidad_victimas,cantidad_victimas_masc,cantidad_victimas_fem,cantidad_victimas_sd,tasa_hechos,tasa_victimas,tasa_victimas_masc,tasa_victimas_fem
0,2,Ciudad Autónoma de Buenos Aires,2000,1,Homicidios dolosos,149,158.0,121.0,37.0,0.0,"4,9750013","5,2755051","8,8257008","2,2783601"
1,2,Ciudad Autónoma de Buenos Aires,2000,2,Homicidios dolosos en grado de tentativa,54,58.0,40.0,18.0,0.0,"1,8030207","1,9365778","2,9175868","1,1083914"
2,2,Ciudad Autónoma de Buenos Aires,2000,3,Muertes en accidentes viales,121,126.0,97.0,29.0,0.0,"4,040102","4,2070484","7,0751481","1,7857418"
3,2,Ciudad Autónoma de Buenos Aires,2000,4,Homicidios culposos por otros hechos,35,36.0,20.0,13.0,3.0,"1,1686245","1,2020137","1,4587934",",80050492"
4,2,Ciudad Autónoma de Buenos Aires,2000,5,Lesiones dolosas,13154,13732.0,8105.0,5485.0,142.0,"439,20248","458,50146","591,17603","337,7515"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21339,94,"Tierra del Fuego, Antártida e Islas del Atlánt...",2022,29_7,Contrabando de elementos nucleares agresivos q...,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
21340,94,"Tierra del Fuego, Antártida e Islas del Atlánt...",2022,29_8,"Otros delitos previstos en leyes especiales n,c,p",3082,NaN,NaN,NaN,NaN,"1693,5648",NaN,NaN,NaN
21341,94,"Tierra del Fuego, Antártida e Islas del Atlánt...",2022,30,Contravenciones,557,NaN,NaN,NaN,NaN,"306,07254",NaN,NaN,NaN
21342,94,"Tierra del Fuego, Antártida e Islas del Atlánt...",2022,31,Suicidios (consumados),22,22.0,12.0,10.0,0.0,"13,150816","13,150816","14,091123","12,175818"


In [106]:
crimes_provinces = df_crimes[["provincia_id", "provincia_nombre"]].copy()

In [107]:
crimes_provinces.drop_duplicates(inplace=True)

In [108]:
crimes_provinces

,provincia_id,provincia_nombre
0,2,Ciudad Autónoma de Buenos Aires
888,6,Buenos Aires
1776,10,Catamarca
2664,14,Córdoba
3552,18,Corrientes
4440,22,Chaco
5336,26,Chubut
6224,30,Entre Ríos
7112,34,Formosa
8000,38,Jujuy


Our INDEC table was

In [109]:
provinces

,id_indec,nombre_indec
0,2,CABA
1,6,Buenos Aires
2,10,Catamarca
3,14,Cordoba
4,18,Corrientes
5,22,Chaco
6,26,Chubut
7,30,Entre Rios
8,34,Formosa
9,38,Jujuy


The ids are the correct, but some of the names are different, they are not abbreviated or they use some special characters.

Since there is a relation 1 to N between PROVINCIA and  DELITO, this table only needs to know the id of their province, using a join it will be able to recover the correct name.

So on, we only need to filter the columns that we will use and rename them conveniently.

In [110]:
del crimes_provinces

In [111]:
df_crimes.columns

Index(['provincia_id', 'provincia_nombre', 'anio', 'codigo_delito_snic_id',
       'codigo_delito_snic_nombre', 'cantidad_hechos', 'cantidad_victimas',
       'cantidad_victimas_masc', 'cantidad_victimas_fem',
       'cantidad_victimas_sd', 'tasa_hechos', 'tasa_victimas',
       'tasa_victimas_masc', 'tasa_victimas_fem'],
      dtype='object')

In [112]:
df_crimes.drop(axis=1, labels=['provincia_nombre'], inplace=True)

In [113]:
df_crimes.columns

Index(['provincia_id', 'anio', 'codigo_delito_snic_id',
       'codigo_delito_snic_nombre', 'cantidad_hechos', 'cantidad_victimas',
       'cantidad_victimas_masc', 'cantidad_victimas_fem',
       'cantidad_victimas_sd', 'tasa_hechos', 'tasa_victimas',
       'tasa_victimas_masc', 'tasa_victimas_fem'],
      dtype='object')

In [114]:
new_names = ["id_provincia_indec", "anio", "id_snic", "nombre_snic", 'cantidad_hechos', 'cantidad_victimas',
       'cantidad_victimas_masc', 'cantidad_victimas_fem',
       'cantidad_victimas_sd', 'tasa_hechos', 'tasa_victimas',
       'tasa_victimas_masc', 'tasa_victimas_fem']

df_crimes.columns = new_names

In [115]:
df_crimes

,id_provincia_indec,anio,id_snic,nombre_snic,cantidad_hechos,cantidad_victimas,cantidad_victimas_masc,cantidad_victimas_fem,cantidad_victimas_sd,tasa_hechos,tasa_victimas,tasa_victimas_masc,tasa_victimas_fem
0,2,2000,1,Homicidios dolosos,149,158.0,121.0,37.0,0.0,"4,9750013","5,2755051","8,8257008","2,2783601"
1,2,2000,2,Homicidios dolosos en grado de tentativa,54,58.0,40.0,18.0,0.0,"1,8030207","1,9365778","2,9175868","1,1083914"
2,2,2000,3,Muertes en accidentes viales,121,126.0,97.0,29.0,0.0,"4,040102","4,2070484","7,0751481","1,7857418"
3,2,2000,4,Homicidios culposos por otros hechos,35,36.0,20.0,13.0,3.0,"1,1686245","1,2020137","1,4587934",",80050492"
4,2,2000,5,Lesiones dolosas,13154,13732.0,8105.0,5485.0,142.0,"439,20248","458,50146","591,17603","337,7515"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21339,94,2022,29_7,Contrabando de elementos nucleares agresivos q...,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
21340,94,2022,29_8,"Otros delitos previstos en leyes especiales n,c,p",3082,NaN,NaN,NaN,NaN,"1693,5648",NaN,NaN,NaN
21341,94,2022,30,Contravenciones,557,NaN,NaN,NaN,NaN,"306,07254",NaN,NaN,NaN
21342,94,2022,31,Suicidios (consumados),22,22.0,12.0,10.0,0.0,"13,150816","13,150816","14,091123","12,175818"


In [116]:
crimes = df_crimes.copy()

In [117]:
del df_crimes

### Suicides

With this one we will need to drop some columns and rename others. Also, we will need to make sure that the departments ids are the INDEC ones.

In [118]:
df_suicides

,id_hecho,federal,provincia_id,provincia_nombre,departamento_id,departamento_nombre,localidad_id,localidad_nombre,anio,mes,...,tipo_lugar_ampliado,tipo_lugar_otro,modalidad,modalidad_ampliado,modalidad_otro,motivo_origen_registro,motivo_origen_registro_otro,suicida_sexo,suicida_tr_edad,suicida_identidad_genero
0,8529,No,42,La Pampa,42002,Norte (General Pico),105030,General Pico,2017,1,...,Vía pública,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar
1,8530,No,42,La Pampa,42003,Sur (General Acha),70030,Guatraché,2017,1,...,Domicilio particular,NaN,Arma de fuego,Arma de fuego,NaN,Intervención policial,NaN,Masculino,55-59,Sin determinar
2,8569,No,78,Santa Cruz,78021,Güer Aike,60,28 de Noviembre,2017,1,...,Vía pública,NaN,Ahorcamiento,Ahorcamiento,NaN,Denuncia particular,NaN,Masculino,15-19,Sin determinar
3,8705,No,58,Neuquén,58035,Confluencia,70,Neuquén,2017,1,...,Hotel/Motel/Hospedaje temporario,Hotel/Motel/Hospedaje temporario,Arma de fuego,Arma de fuego,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar
4,8706,No,58,Neuquén,58035,Confluencia,100,Plottier,2017,1,...,Domicilio particular,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21714,37640,No,18,Corrientes,18049,Esquina,10,Esquina,2022,7,...,Cárcel o comisaría,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,30-34,Varón
21715,37642,No,18,Corrientes,18070,Goya,20,Goya,2022,9,...,Domicilio particular,NaN,Sin determinar,Sin determinar,NaN,Intervención policial,NaN,Masculino,50-54,Varón
21716,37643,No,18,Corrientes,18070,Goya,20,Goya,2022,9,...,Domicilio particular,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,15-19,Varón
21717,37639,No,18,Corrientes,18133,San Cosme,30,San Cosme,2022,7,...,Campo/descampado/zona rural,Campo/descampado/zona rural,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,30-34,Varón


In [119]:
suicides_departments = df_suicides[["departamento_id", "departamento_nombre"]].copy()

In [120]:
suicides_departments.drop_duplicates(inplace=True)

In [121]:
suicides_departments

,departamento_id,departamento_nombre
0,42002,Norte (General Pico)
1,42003,Sur (General Acha)
2,78021,Güer Aike
3,58035,Confluencia
6,58049,Huiliches
...,...,...
18600,10056,El Alto
18800,38098,Tumbaya
19327,10028,Antofagasta de la Sierra
19422,58999,Departamento sin determinar


In [122]:
departments #the indec based ones

,id_indec,nombre_indec,id_provincia_indec
0,2000,CABA,2
1,6007,Adolfo Alsina,6
2,6014,Adolfo Gonzales Chaves,6
3,6021,Alberti,6
4,6028,Almirante Brown,6
...,...,...,...
505,90105,Tafí Viejo,90
506,90112,Trancas,90
507,90119,Yerba Buena,90
508,94007,Río Grande/Tolhuin,94


The suicide based ones are 505, while the INDEC's are 510. This can be because there isn´t any register for some of the departments. However, we have to check that the ones that appear are the correct ones.

In [123]:
suicides_departments.columns = ["id_indec", "nombre_indec"]

In [124]:
merged_departments = pd.merge(suicides_departments, departments, on="id_indec", how="left")

In [125]:
merged_departments

,id_indec,nombre_indec_x,nombre_indec_y,id_provincia_indec
0,42002,Norte (General Pico),NaN,NaN
1,42003,Sur (General Acha),NaN,NaN
2,78021,Güer Aike,Güer Aike,78.0
3,58035,Confluencia,Confluencia,58.0
4,58049,Huiliches,Huiliches,58.0
...,...,...,...,...
500,10056,El Alto,El Alto,10.0
501,38098,Tumbaya,Tumbaya,38.0
502,10028,Antofagasta de la Sierra,Antofagasta de la Sierra,10.0
503,58999,Departamento sin determinar,NaN,NaN


We merged the tables x (suicides_departments) and y (departments) with this kind of pandas left join.

We can see that some names that are from the suicides table doesn´t correspond to any of the INDEC's registers. Therefore, these one have NaN values in the `y` columns.

How frecuently does this happen?

In [126]:
incorrect_pairs = merged_departments[merged_departments["nombre_indec_x"] != merged_departments["nombre_indec_y"]]

In [127]:
incorrect_pairs

,id_indec,nombre_indec_x,nombre_indec_y,id_provincia_indec
0,42002,Norte (General Pico),NaN,NaN
1,42003,Sur (General Acha),NaN,NaN
8,66056,Grl. José de San Martín,General José de San Martín,66.0
12,66035,Cerrillos,Cerrillos,66.0
17,58112,Zapala,Zapala,58.0
...,...,...,...,...
468,22126,1º de Mayo,1° de Mayo,22.0
478,46056,General Angel V. Peñaloza,Ángel Vicente Peñaloza,46.0
485,46119,San Blas de los Sauces,San Blas de Los Sauces,46.0
495,46049,Famatina,Famatina,46.0


We have 66 over 505 rows with problems. However, it seems like some of them, the ones that doesn´t have a `NaN` value in the `y` column, have a pretty correct name, it differs just a little from the original ones.

As these are not really a lot of rows, let´s see this directly

In [128]:
incorrect_pairs.head(66)

,id_indec,nombre_indec_x,nombre_indec_y,id_provincia_indec
0,42002,Norte (General Pico),NaN,NaN
1,42003,Sur (General Acha),NaN,NaN
8,66056,Grl. José de San Martín,General José de San Martín,66.0
12,66035,Cerrillos,Cerrillos,66.0
17,58112,Zapala,Zapala,58.0
...,...,...,...,...
468,22126,1º de Mayo,1° de Mayo,22.0
478,46056,General Angel V. Peñaloza,Ángel Vicente Peñaloza,46.0
485,46119,San Blas de los Sauces,San Blas de Los Sauces,46.0
495,46049,Famatina,Famatina,46.0


It looks that is the case. The departments that doesn´t have a `NaN` value in the `y` column are valid ones, it just that some of the suicide's table ones have accents, white spices or typos like "Gral." in replacement for "General". We can simply preserve the merged INDEC names in these cases.

Now, what we can do with the `NaN` ones?


In [129]:
merged_departments["nombre_indec_y"].isna().sum()

31

There are 31 of them. However, some are "Departamento sin determinar and other "Comuna N". There is an INDEC id for the non-determinated departments and a "Comuna" one means that it belongs to CABA. Therefore, without this ones, the incorrect pairs would be:

In [130]:
incorrect_pairs = incorrect_pairs[incorrect_pairs["nombre_indec_y"].isna()]

In [131]:
incorrect_pairs = incorrect_pairs[incorrect_pairs["nombre_indec_x"] != "Departamento sin determinar"]

In [132]:
incorrect_pairs = incorrect_pairs[incorrect_pairs.apply(lambda x: not ("Comuna" in x['nombre_indec_x']), axis=1)]

In [133]:
incorrect_pairs

,id_indec,nombre_indec_x,nombre_indec_y,id_provincia_indec
0,42002,Norte (General Pico),NaN,NaN
1,42003,Sur (General Acha),NaN,NaN
23,66077,La Caldera,NaN,NaN
80,42001,Centro (Santa Rosa),NaN,NaN
82,42004,Oeste (25 de Mayo),NaN,NaN
342,6466,Lezama,NaN,NaN


So this are really the only problematic departments.

However, it seems like the ones that uses parentesis are just a more specific description of those. For example, we can check that `Santa Rosa` and `25 de Mayo` really exist.

For `La Caldera` and `Lezama` we don´t have insight on what we can do. But as these are just a few rows, we can see if our INDEC tables has the corresponding values.

In [134]:
searched_departments = ["General Pico", "General Acha", "Santa Rosa", "25 de Mayo"]

departments[departments["nombre_indec"].isin(searched_departments)]

,id_indec,nombre_indec,id_provincia_indec
130,6854,25 de Mayo,6
148,10098,Santa Rosa,10
226,22168,25 de Mayo,22
339,50112,Santa Rosa,50
358,54119,25 de Mayo,54
387,62091,25 de Mayo,62
427,70126,25 de Mayo,70


There are multiple `Santa Rosa` and `25 de Mayo` locations, depending on the province. But our dataset has only one possibility for each, so cases that are from one of these have to share the same province.

In [135]:
df_suicides[df_suicides["departamento_id"].isin([42001, 42004])]

,id_hecho,federal,provincia_id,provincia_nombre,departamento_id,departamento_nombre,localidad_id,localidad_nombre,anio,mes,...,tipo_lugar_ampliado,tipo_lugar_otro,modalidad,modalidad_ampliado,modalidad_otro,motivo_origen_registro,motivo_origen_registro_otro,suicida_sexo,suicida_tr_edad,suicida_identidad_genero
418,9221,No,42,La Pampa,42001,Centro (Santa Rosa),21020,Santa Rosa,2017,7,...,Domicilio particular,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Femenino,20-24,Sin determinar
419,9222,No,42,La Pampa,42001,Centro (Santa Rosa),7030,Miguel Riglos,2017,7,...,Domicilio particular,NaN,Arma de fuego,Arma de fuego,NaN,Intervención policial,NaN,Masculino,65-69,Sin determinar
420,9219,No,42,La Pampa,42001,Centro (Santa Rosa),140020,Toay,2017,7,...,Domicilio particular,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,35-39,Sin determinar
421,9220,No,42,La Pampa,42001,Centro (Santa Rosa),21020,Santa Rosa,2017,7,...,Domicilio particular,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,30-34,Sin determinar
428,9229,No,42,La Pampa,42001,Centro (Santa Rosa),21020,Santa Rosa,2017,8,...,Domicilio particular,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Femenino,70-74,Sin determinar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19305,34974,No,42,La Pampa,42001,Centro (Santa Rosa),21020,Santa Rosa,2022,12,...,Domicilio particular,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Femenino,90 y más,Mujer
19307,34976,No,42,La Pampa,42001,Centro (Santa Rosa),21020,Santa Rosa,2022,12,...,Domicilio particular,NaN,Ahorcamiento,Ahorcamiento,NaN,Denuncia particular,NaN,Femenino,25-29,Mujer
19308,34977,No,42,La Pampa,42001,Centro (Santa Rosa),21020,Santa Rosa,2022,12,...,Domicilio particular,NaN,Ahorcamiento,Ahorcamiento,NaN,Denuncia particular,NaN,Masculino,70-74,Varón
19309,34978,No,42,La Pampa,42001,Centro (Santa Rosa),98050,Victorica,2022,12,...,Domicilio particular,NaN,Envenenamiento,Envenenamiento,NaN,Intervención policial,NaN,Femenino,50-54,Mujer


Both are from `La Pampa`, which has the `42` INDEC id. However, none of our departments has a province id of `42`.

In [136]:
departments[departments["id_provincia_indec"] == 42]

,id_indec,nombre_indec,id_provincia_indec
284,42007,Atreucó,42
285,42014,Caleu Caleu,42
286,42021,Capital,42
287,42028,Catriló,42
288,42035,Conhelo,42
289,42042,Curacó,42
290,42049,Chalileo,42
291,42056,Chapaleufú,42
292,42063,Chical Co,42
293,42070,Guatraché,42


We don´t have `Santa Rosa` or `25 de Mayo` departments in `La Pampa` for our INDEC table. We migth discard these rows.

Now, with all of this in mind, we can clean the original suicides table

In [137]:
comunas_rows = df_suicides[df_suicides.apply(lambda x: "Comuna" in x["departamento_nombre"], axis=1)]

In [138]:
comunas_ids = comunas_rows["departamento_id"].unique()

In [139]:
comunas_ids

array([2005, 2007, 2008, 2009, 2013, 2014, 2001, 2003, 2011, 2004, 2006,
       2015, 2010, 2012, 2002])

In [140]:
departments

,id_indec,nombre_indec,id_provincia_indec
0,2000,CABA,2
1,6007,Adolfo Alsina,6
2,6014,Adolfo Gonzales Chaves,6
3,6021,Alberti,6
4,6028,Almirante Brown,6
...,...,...,...
505,90105,Tafí Viejo,90
506,90112,Trancas,90
507,90119,Yerba Buena,90
508,94007,Río Grande/Tolhuin,94


Gonna replace those ids with `2000`

In [141]:
df_suicides["departamento_id"] = df_suicides["departamento_id"].replace(to_replace=comunas_ids, value=2000)

In [142]:
df_suicides

,id_hecho,federal,provincia_id,provincia_nombre,departamento_id,departamento_nombre,localidad_id,localidad_nombre,anio,mes,...,tipo_lugar_ampliado,tipo_lugar_otro,modalidad,modalidad_ampliado,modalidad_otro,motivo_origen_registro,motivo_origen_registro_otro,suicida_sexo,suicida_tr_edad,suicida_identidad_genero
0,8529,No,42,La Pampa,42002,Norte (General Pico),105030,General Pico,2017,1,...,Vía pública,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar
1,8530,No,42,La Pampa,42003,Sur (General Acha),70030,Guatraché,2017,1,...,Domicilio particular,NaN,Arma de fuego,Arma de fuego,NaN,Intervención policial,NaN,Masculino,55-59,Sin determinar
2,8569,No,78,Santa Cruz,78021,Güer Aike,60,28 de Noviembre,2017,1,...,Vía pública,NaN,Ahorcamiento,Ahorcamiento,NaN,Denuncia particular,NaN,Masculino,15-19,Sin determinar
3,8705,No,58,Neuquén,58035,Confluencia,70,Neuquén,2017,1,...,Hotel/Motel/Hospedaje temporario,Hotel/Motel/Hospedaje temporario,Arma de fuego,Arma de fuego,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar
4,8706,No,58,Neuquén,58035,Confluencia,100,Plottier,2017,1,...,Domicilio particular,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21714,37640,No,18,Corrientes,18049,Esquina,10,Esquina,2022,7,...,Cárcel o comisaría,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,30-34,Varón
21715,37642,No,18,Corrientes,18070,Goya,20,Goya,2022,9,...,Domicilio particular,NaN,Sin determinar,Sin determinar,NaN,Intervención policial,NaN,Masculino,50-54,Varón
21716,37643,No,18,Corrientes,18070,Goya,20,Goya,2022,9,...,Domicilio particular,NaN,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,15-19,Varón
21717,37639,No,18,Corrientes,18133,San Cosme,30,San Cosme,2022,7,...,Campo/descampado/zona rural,Campo/descampado/zona rural,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,30-34,Varón


In [143]:
df_suicides.columns = ['id_hecho', 'federal', 'provincia_id', 'provincia_nombre',
       'id_indec', 'departamento_nombre', 'localidad_id',
       'localidad_nombre', 'anio', 'mes', 'fecha_hecho', 'hora_hecho',
       'tipo_lugar', 'tipo_lugar_ampliado', 'tipo_lugar_otro', 'modalidad',
       'modalidad_ampliado', 'modalidad_otro', 'motivo_origen_registro',
       'motivo_origen_registro_otro', 'suicida_sexo', 'suicida_tr_edad',
       'suicida_identidad_genero']

In [144]:
df_suicides_merged = pd.merge(df_suicides, departments, on="id_indec", how="left")

In [145]:
df_suicides_merged

,id_hecho,federal,provincia_id,provincia_nombre,id_indec,departamento_nombre,localidad_id,localidad_nombre,anio,mes,...,modalidad,modalidad_ampliado,modalidad_otro,motivo_origen_registro,motivo_origen_registro_otro,suicida_sexo,suicida_tr_edad,suicida_identidad_genero,nombre_indec,id_provincia_indec
0,8529,No,42,La Pampa,42002,Norte (General Pico),105030,General Pico,2017,1,...,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar,NaN,NaN
1,8530,No,42,La Pampa,42003,Sur (General Acha),70030,Guatraché,2017,1,...,Arma de fuego,Arma de fuego,NaN,Intervención policial,NaN,Masculino,55-59,Sin determinar,NaN,NaN
2,8569,No,78,Santa Cruz,78021,Güer Aike,60,28 de Noviembre,2017,1,...,Ahorcamiento,Ahorcamiento,NaN,Denuncia particular,NaN,Masculino,15-19,Sin determinar,Güer Aike,78.0
3,8705,No,58,Neuquén,58035,Confluencia,70,Neuquén,2017,1,...,Arma de fuego,Arma de fuego,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar,Confluencia,58.0
4,8706,No,58,Neuquén,58035,Confluencia,100,Plottier,2017,1,...,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar,Confluencia,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21714,37640,No,18,Corrientes,18049,Esquina,10,Esquina,2022,7,...,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,30-34,Varón,Esquina,18.0
21715,37642,No,18,Corrientes,18070,Goya,20,Goya,2022,9,...,Sin determinar,Sin determinar,NaN,Intervención policial,NaN,Masculino,50-54,Varón,Goya,18.0
21716,37643,No,18,Corrientes,18070,Goya,20,Goya,2022,9,...,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,15-19,Varón,Goya,18.0
21717,37639,No,18,Corrientes,18133,San Cosme,30,San Cosme,2022,7,...,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,30-34,Varón,San Cosme,18.0


In [146]:
suicide_cleaned_table = df_suicides_merged[~(df_suicides_merged["nombre_indec"].isna())].copy()

In [147]:
suicide_cleaned_table

,id_hecho,federal,provincia_id,provincia_nombre,id_indec,departamento_nombre,localidad_id,localidad_nombre,anio,mes,...,modalidad,modalidad_ampliado,modalidad_otro,motivo_origen_registro,motivo_origen_registro_otro,suicida_sexo,suicida_tr_edad,suicida_identidad_genero,nombre_indec,id_provincia_indec
2,8569,No,78,Santa Cruz,78021,Güer Aike,60,28 de Noviembre,2017,1,...,Ahorcamiento,Ahorcamiento,NaN,Denuncia particular,NaN,Masculino,15-19,Sin determinar,Güer Aike,78.0
3,8705,No,58,Neuquén,58035,Confluencia,70,Neuquén,2017,1,...,Arma de fuego,Arma de fuego,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar,Confluencia,58.0
4,8706,No,58,Neuquén,58035,Confluencia,100,Plottier,2017,1,...,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar,Confluencia,58.0
5,8707,No,58,Neuquén,58035,Confluencia,40,Cutral Có,2017,1,...,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,20-24,Sin determinar,Confluencia,58.0
6,8708,No,58,Neuquén,58049,Huiliches,10,Junín de los Andes,2017,1,...,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,45-49,Sin determinar,Huiliches,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21713,38831,No,14,Córdoba,14021,Colón,240,Saldán,2022,8,...,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Femenino,15-19,Mujer,Colón,14.0
21714,37640,No,18,Corrientes,18049,Esquina,10,Esquina,2022,7,...,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,30-34,Varón,Esquina,18.0
21715,37642,No,18,Corrientes,18070,Goya,20,Goya,2022,9,...,Sin determinar,Sin determinar,NaN,Intervención policial,NaN,Masculino,50-54,Varón,Goya,18.0
21716,37643,No,18,Corrientes,18070,Goya,20,Goya,2022,9,...,Ahorcamiento,Ahorcamiento,NaN,Intervención policial,NaN,Masculino,15-19,Varón,Goya,18.0


`21426` of the `21719` rows, that corresponds to 98.65% of the data original data retained. We can continue with this and assume a really small loss.

All the `departamento_id` values correspond to valid INDEC ones, we can drop the provinces (they can be recovered using the departments) and other columns that are not longer useful.

In [148]:
suicide_cleaned_table.columns

Index(['id_hecho', 'federal', 'provincia_id', 'provincia_nombre', 'id_indec',
       'departamento_nombre', 'localidad_id', 'localidad_nombre', 'anio',
       'mes', 'fecha_hecho', 'hora_hecho', 'tipo_lugar', 'tipo_lugar_ampliado',
       'tipo_lugar_otro', 'modalidad', 'modalidad_ampliado', 'modalidad_otro',
       'motivo_origen_registro', 'motivo_origen_registro_otro', 'suicida_sexo',
       'suicida_tr_edad', 'suicida_identidad_genero', 'nombre_indec',
       'id_provincia_indec'],
      dtype='object')

In [149]:
useful_cols = ['id_hecho', 'id_indec', 'fecha_hecho', 'hora_hecho', 'tipo_lugar', 'modalidad',
       'motivo_origen_registro', 'suicida_sexo',
       'suicida_tr_edad', 'suicida_identidad_genero']

In [150]:
suicide_cleaned_table = suicide_cleaned_table[useful_cols]

In [151]:
suicide_cleaned_table

,id_hecho,id_indec,fecha_hecho,hora_hecho,tipo_lugar,modalidad,motivo_origen_registro,suicida_sexo,suicida_tr_edad,suicida_identidad_genero
2,8569,78021,01-01-2017,19:00:00,Vía pública,Ahorcamiento,Denuncia particular,Masculino,15-19,Sin determinar
3,8705,58035,31-01-2017,17:30:00,Otro Lugar ( Especificar ),Arma de fuego,Intervención policial,Masculino,20-24,Sin determinar
4,8706,58035,31-01-2017,22:40:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,20-24,Sin determinar
5,8707,58035,29-01-2017,20:30:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,20-24,Sin determinar
6,8708,58049,02-01-2017,14:25:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,45-49,Sin determinar
...,...,...,...,...,...,...,...,...,...,...
21713,38831,14021,28-08-2022,11:11:11,Domicilio particular,Ahorcamiento,Intervención policial,Femenino,15-19,Mujer
21714,37640,18049,09-07-2022,07:30:00,Cárcel o comisaría,Ahorcamiento,Intervención policial,Masculino,30-34,Varón
21715,37642,18070,11-09-2022,05:00:00,Domicilio particular,Sin determinar,Intervención policial,Masculino,50-54,Varón
21716,37643,18070,27-09-2022,06:45:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,15-19,Varón


Now we need to rename them.

In [152]:
new_col_names = ["id_hecho", "id_departamento_indec", "fecha", "hora", "lugar",
                 "modalidad", "origen_registro", "suicida_sexo", "suicida_rango_edad", "suicida_identidad_genero"]

In [153]:
suicide_cleaned_table.columns = new_col_names

In [154]:
suicide_cleaned_table

,id_hecho,id_departamento_indec,fecha,hora,lugar,modalidad,origen_registro,suicida_sexo,suicida_rango_edad,suicida_identidad_genero
2,8569,78021,01-01-2017,19:00:00,Vía pública,Ahorcamiento,Denuncia particular,Masculino,15-19,Sin determinar
3,8705,58035,31-01-2017,17:30:00,Otro Lugar ( Especificar ),Arma de fuego,Intervención policial,Masculino,20-24,Sin determinar
4,8706,58035,31-01-2017,22:40:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,20-24,Sin determinar
5,8707,58035,29-01-2017,20:30:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,20-24,Sin determinar
6,8708,58049,02-01-2017,14:25:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,45-49,Sin determinar
...,...,...,...,...,...,...,...,...,...,...
21713,38831,14021,28-08-2022,11:11:11,Domicilio particular,Ahorcamiento,Intervención policial,Femenino,15-19,Mujer
21714,37640,18049,09-07-2022,07:30:00,Cárcel o comisaría,Ahorcamiento,Intervención policial,Masculino,30-34,Varón
21715,37642,18070,11-09-2022,05:00:00,Domicilio particular,Sin determinar,Intervención policial,Masculino,50-54,Varón
21716,37643,18070,27-09-2022,06:45:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,15-19,Varón


In [155]:
suicides = suicide_cleaned_table.copy()

In [156]:
del df_suicides, suicide_cleaned_table, new_col_names, merged_departments, useful_cols, df_suicides_merged, comunas_ids, comunas_rows, incorrect_pairs, suicides_departments

In [157]:
suicides

,id_hecho,id_departamento_indec,fecha,hora,lugar,modalidad,origen_registro,suicida_sexo,suicida_rango_edad,suicida_identidad_genero
2,8569,78021,01-01-2017,19:00:00,Vía pública,Ahorcamiento,Denuncia particular,Masculino,15-19,Sin determinar
3,8705,58035,31-01-2017,17:30:00,Otro Lugar ( Especificar ),Arma de fuego,Intervención policial,Masculino,20-24,Sin determinar
4,8706,58035,31-01-2017,22:40:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,20-24,Sin determinar
5,8707,58035,29-01-2017,20:30:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,20-24,Sin determinar
6,8708,58049,02-01-2017,14:25:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,45-49,Sin determinar
...,...,...,...,...,...,...,...,...,...,...
21713,38831,14021,28-08-2022,11:11:11,Domicilio particular,Ahorcamiento,Intervención policial,Femenino,15-19,Mujer
21714,37640,18049,09-07-2022,07:30:00,Cárcel o comisaría,Ahorcamiento,Intervención policial,Masculino,30-34,Varón
21715,37642,18070,11-09-2022,05:00:00,Domicilio particular,Sin determinar,Intervención policial,Masculino,50-54,Varón
21716,37643,18070,27-09-2022,06:45:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,15-19,Varón


That should be enough for this dataset by now.

### Mean Income

The only thing that we have to validate here, at the beginning, is that the indec departments are correct. We can see that there are a bunch on `NaN`s just printing the head and tail of the dataframe.

In [158]:
df_mean_income

,fecha,codigo_departamento_indec,id_provincia_indec,clae2,w_mean
0,2014-01-01,2000.0,2.0,1,13085
1,2014-01-01,2000.0,2.0,2,10081
2,2014-01-01,2000.0,2.0,3,17837
3,2014-01-01,2000.0,2.0,5,57688
4,2014-01-01,2000.0,2.0,6,62211
...,...,...,...,...,...
3597476,2023-09-01,NaN,NaN,93,247445
3597477,2023-09-01,NaN,NaN,94,363199
3597478,2023-09-01,NaN,NaN,95,196750
3597479,2023-09-01,NaN,NaN,96,231747


In [160]:
df_mean_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3597481 entries, 0 to 3597480
Data columns (total 5 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   fecha                      object 
 1   codigo_departamento_indec  float64
 2   id_provincia_indec         float64
 3   clae2                      int64  
 4   w_mean                     int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 137.2+ MB


Both the province and departments ids are using float64 datatype, it should be an int.

In [161]:
df_mean_income["codigo_departamento_indec"].unique()

array([ 2000.,  6007.,  6014.,  6021.,  6028.,  6035.,  6042.,  6049.,
        6056.,  6063.,  6070.,  6077.,  6084.,  6091.,  6098.,  6105.,
        6112.,  6119.,  6126.,  6134.,  6140.,  6147.,  6154.,  6161.,
        6168.,  6175.,  6182.,  6189.,  6196.,  6203.,  6210.,  6217.,
        6224.,  6231.,  6238.,  6245.,  6252.,  6260.,  6266.,  6270.,
        6274.,  6277.,  6280.,  6287.,  6294.,  6301.,  6308.,  6315.,
        6322.,  6329.,  6336.,  6343.,  6351.,  6357.,  6364.,  6371.,
        6385.,  6392.,  6399.,  6406.,  6408.,  6410.,  6412.,  6413.,
        6420.,  6427.,  6434.,  6441.,  6448.,  6455.,  6462.,  6469.,
        6476.,  6483.,  6490.,  6497.,  6505.,  6511.,  6515.,  6518.,
        6525.,  6532.,  6539.,  6547.,  6553.,  6560.,  6568.,  6574.,
        6581.,  6588.,  6595.,  6602.,  6609.,  6616.,  6623.,  6630.,
        6638.,  6644.,  6648.,  6651.,  6655.,  6658.,  6665.,  6672.,
        6679.,  6686.,  6693.,  6700.,  6707.,  6714.,  6721.,  6728.,
      

In [171]:
df_mean_income[df_mean_income["codigo_departamento_indec"].isna()]

,fecha,codigo_departamento_indec,id_provincia_indec,clae2,w_mean
30910,2014-01-01,NaN,NaN,1,4933
30911,2014-01-01,NaN,NaN,2,2641
30912,2014-01-01,NaN,NaN,3,15357
30913,2014-01-01,NaN,NaN,6,41990
30914,2014-01-01,NaN,NaN,7,20778
...,...,...,...,...,...
3597476,2023-09-01,NaN,NaN,93,247445
3597477,2023-09-01,NaN,NaN,94,363199
3597478,2023-09-01,NaN,NaN,95,196750
3597479,2023-09-01,NaN,NaN,96,231747


These can be problematic, we will focus on the ones that are not `NaN`

In [175]:
df_mean_income.dropna(inplace=True)

In [176]:
df_mean_income

,fecha,codigo_departamento_indec,id_provincia_indec,clae2,w_mean
0,2014-01-01,2000.0,2.0,1,13085
1,2014-01-01,2000.0,2.0,2,10081
2,2014-01-01,2000.0,2.0,3,17837
3,2014-01-01,2000.0,2.0,5,57688
4,2014-01-01,2000.0,2.0,6,62211
...,...,...,...,...,...
3597392,2023-09-01,94014.0,94.0,93,536119
3597393,2023-09-01,94014.0,94.0,94,504914
3597394,2023-09-01,94014.0,94.0,95,367788
3597395,2023-09-01,94014.0,94.0,96,316829


3587653 of the 3597481 rows preserved, that is 99.73% of the data. We will need to check about the completness of the dataset then, because if it was a full year that was dropped, then it could be problematic.

In [178]:
df_mean_income[["codigo_departamento_indec", "id_provincia_indec"]] = df_mean_income[["codigo_departamento_indec", "id_provincia_indec"]].astype(int)

In [181]:
df_mean_income

,fecha,id_indec,id_provincia_indec,clae2,w_mean
0,2014-01-01,2000,2,1,13085
1,2014-01-01,2000,2,2,10081
2,2014-01-01,2000,2,3,17837
3,2014-01-01,2000,2,5,57688
4,2014-01-01,2000,2,6,62211
...,...,...,...,...,...
3597392,2023-09-01,94014,94,93,536119
3597393,2023-09-01,94014,94,94,504914
3597394,2023-09-01,94014,94,95,367788
3597395,2023-09-01,94014,94,96,316829


In [180]:
df_mean_income.rename(columns={"codigo_departamento_indec":"id_indec"},inplace=True)

In [182]:
merged_income = pd.merge(df_mean_income, departments, on="id_indec", how="left")

In [183]:
merged_income

,fecha,id_indec,id_provincia_indec_x,clae2,w_mean,nombre_indec,id_provincia_indec_y
0,2014-01-01,2000,2,1,13085,CABA,2
1,2014-01-01,2000,2,2,10081,CABA,2
2,2014-01-01,2000,2,3,17837,CABA,2
3,2014-01-01,2000,2,5,57688,CABA,2
4,2014-01-01,2000,2,6,62211,CABA,2
...,...,...,...,...,...,...,...
3587648,2023-09-01,94014,94,93,536119,Ushuaia,94
3587649,2023-09-01,94014,94,94,504914,Ushuaia,94
3587650,2023-09-01,94014,94,95,367788,Ushuaia,94
3587651,2023-09-01,94014,94,96,316829,Ushuaia,94


In [184]:
merged_income.isna().sum()

fecha                   0
id_indec                0
id_provincia_indec_x    0
clae2                   0
w_mean                  0
nombre_indec            0
id_provincia_indec_y    0
dtype: int64

Everything is good, let´s just select the columns that we are going to use and it will be done.

In [187]:
useful_cols = ['fecha', 'id_indec', 'clae2', 'w_mean']

In [191]:
mean_income = df_mean_income[useful_cols].copy()

In [192]:
mean_income.columns = ["fecha", "id_departamento_indec", "clae2", "w_mean"]

In [193]:
mean_income

,fecha,id_departamento_indec,clae2,w_mean
0,2014-01-01,2000,1,13085
1,2014-01-01,2000,2,10081
2,2014-01-01,2000,3,17837
3,2014-01-01,2000,5,57688
4,2014-01-01,2000,6,62211
...,...,...,...,...
3597392,2023-09-01,94014,93,536119
3597393,2023-09-01,94014,94,504914
3597394,2023-09-01,94014,95,367788
3597395,2023-09-01,94014,96,316829


In [194]:
del df_mean_income, useful_cols, merged_income

### CLAE dictionary

We just need to select the columns that we will use.

In [195]:
df_clae2

,clae2,clae2_desc,letra,letra_desc
0,1,"Agricultura, ganadería, caza y servicios relac...",A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y ..."
1,2,Silvicultura y explotación forestal,A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y ..."
2,3,Pesca y acuicultura,A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y ..."
3,5,Extracción de carbón y lignito,B,EXPLOTACION DE MINAS Y CANTERAS
4,6,Extracción de petróleo crudo y gas natural,B,EXPLOTACION DE MINAS Y CANTERAS
...,...,...,...,...
81,93,"Actividades deportivas, recreativas y de entre...",R,"SERVICIOS ARTÍSTICOS, CULTURALES, DEPORTIVOS..."
82,94,Servicios de asociaciones,S,SERVICIOS DE ASOCIACIONES Y SERVICIOS PERSONA...
83,95,Reparación de computadoras y equipos de uso do...,S,SERVICIOS DE ASOCIACIONES Y SERVICIOS PERSONA...
84,96,Otros servicios personales,S,SERVICIOS DE ASOCIACIONES Y SERVICIOS PERSONA...


In [196]:
df_clae2 = df_clae2[["clae2", "clae2_desc"]].drop_duplicates()

In [197]:
df_clae2

,clae2,clae2_desc
0,1,"Agricultura, ganadería, caza y servicios relac..."
1,2,Silvicultura y explotación forestal
2,3,Pesca y acuicultura
3,5,Extracción de carbón y lignito
4,6,Extracción de petróleo crudo y gas natural
...,...,...
81,93,"Actividades deportivas, recreativas y de entre..."
82,94,Servicios de asociaciones
83,95,Reparación de computadoras y equipos de uso do...
84,96,Otros servicios personales


In [198]:
clae2 = df_clae2.copy()

In [199]:
del df_clae2

## Checking out inconsistencies in each dataset

We mapped our datasets to the model that follows ERD that we saw before, now we have to check that the data is useful on each of them. Eliminate typos, handle missing values and so on.

### Provinces

In [201]:
provinces

,id_indec,nombre_indec
0,2,CABA
1,6,Buenos Aires
2,10,Catamarca
3,14,Cordoba
4,18,Corrientes
5,22,Chaco
6,26,Chubut
7,30,Entre Rios
8,34,Formosa
9,38,Jujuy


In [202]:
provinces.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   (id_indec,)      24 non-null     int64 
 1   (nombre_indec,)  24 non-null     object
dtypes: int64(1), object(1)
memory usage: 512.0+ bytes


Nothing to change here.

### Departments

In [203]:
departments

,id_indec,nombre_indec,id_provincia_indec
0,2000,CABA,2
1,6007,Adolfo Alsina,6
2,6014,Adolfo Gonzales Chaves,6
3,6021,Alberti,6
4,6028,Almirante Brown,6
...,...,...,...
505,90105,Tafí Viejo,90
506,90112,Trancas,90
507,90119,Yerba Buena,90
508,94007,Río Grande/Tolhuin,94


In [204]:
departments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id_indec            510 non-null    int64 
 1   nombre_indec        510 non-null    object
 2   id_provincia_indec  510 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 12.1+ KB


Everything is ok.

### Crimes

In [205]:
crimes

,id_provincia_indec,anio,id_snic,nombre_snic,cantidad_hechos,cantidad_victimas,cantidad_victimas_masc,cantidad_victimas_fem,cantidad_victimas_sd,tasa_hechos,tasa_victimas,tasa_victimas_masc,tasa_victimas_fem
0,2,2000,1,Homicidios dolosos,149,158.0,121.0,37.0,0.0,"4,9750013","5,2755051","8,8257008","2,2783601"
1,2,2000,2,Homicidios dolosos en grado de tentativa,54,58.0,40.0,18.0,0.0,"1,8030207","1,9365778","2,9175868","1,1083914"
2,2,2000,3,Muertes en accidentes viales,121,126.0,97.0,29.0,0.0,"4,040102","4,2070484","7,0751481","1,7857418"
3,2,2000,4,Homicidios culposos por otros hechos,35,36.0,20.0,13.0,3.0,"1,1686245","1,2020137","1,4587934",",80050492"
4,2,2000,5,Lesiones dolosas,13154,13732.0,8105.0,5485.0,142.0,"439,20248","458,50146","591,17603","337,7515"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21339,94,2022,29_7,Contrabando de elementos nucleares agresivos q...,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
21340,94,2022,29_8,"Otros delitos previstos en leyes especiales n,c,p",3082,NaN,NaN,NaN,NaN,"1693,5648",NaN,NaN,NaN
21341,94,2022,30,Contravenciones,557,NaN,NaN,NaN,NaN,"306,07254",NaN,NaN,NaN
21342,94,2022,31,Suicidios (consumados),22,22.0,12.0,10.0,0.0,"13,150816","13,150816","14,091123","12,175818"


In [206]:
crimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id_provincia_indec      21344 non-null  int64  
 1   anio                    21344 non-null  int64  
 2   id_snic                 21344 non-null  object 
 3   nombre_snic             21344 non-null  object 
 4   cantidad_hechos         21344 non-null  int64  
 5   cantidad_victimas       7608 non-null   float64
 6   cantidad_victimas_masc  7608 non-null   float64
 7   cantidad_victimas_fem   7608 non-null   float64
 8   cantidad_victimas_sd    7608 non-null   float64
 9   tasa_hechos             21344 non-null  object 
 10  tasa_victimas           7608 non-null   object 
 11  tasa_victimas_masc      7608 non-null   object 
 12  tasa_victimas_fem       7608 non-null   object 
dtypes: float64(4), int64(3), object(6)
memory usage: 2.1+ MB


There is missing data in the `victims` related columns. As is is the same for each type, this would probably be because there are some crimes that doesn´t have any registered victim. Therefore, there won´t be registers for their sex or the proportions (`tasa` related columns)

In [210]:
missing_val_crimes = crimes[crimes["cantidad_victimas"].isna()]

In [211]:
missing_val_crimes

,id_provincia_indec,anio,id_snic,nombre_snic,cantidad_hechos,cantidad_victimas,cantidad_victimas_masc,cantidad_victimas_fem,cantidad_victimas_sd,tasa_hechos,tasa_victimas,tasa_victimas_masc,tasa_victimas_fem
11,2,2000,12,Delitos contra el estado civil,1,NaN,NaN,NaN,NaN,",03338927",NaN,NaN,NaN
12,2,2000,13,Amenazas,11034,NaN,NaN,NaN,NaN,"368,41724",NaN,NaN,NaN
14,2,2000,15,Robos (excluye los agravados por el resultado ...,68937,NaN,NaN,NaN,NaN,"2301,7561",NaN,NaN,NaN
15,2,2000,16,Tentativas de robo (excluye las agravadas por ...,2620,NaN,NaN,NaN,NaN,"87,479889",NaN,NaN,NaN
16,2,2000,17,Robos agravados por el resultado de lesiones y...,37,NaN,NaN,NaN,NaN,"1,2354031",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21338,94,2022,29_6,Contrabando agravado,23,NaN,NaN,NaN,NaN,"12,638543",NaN,NaN,NaN
21339,94,2022,29_7,Contrabando de elementos nucleares agresivos q...,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
21340,94,2022,29_8,"Otros delitos previstos en leyes especiales n,c,p",3082,NaN,NaN,NaN,NaN,"1693,5648",NaN,NaN,NaN
21341,94,2022,30,Contravenciones,557,NaN,NaN,NaN,NaN,"306,07254",NaN,NaN,NaN


In [213]:
missing_val_crimes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13736 entries, 11 to 21343
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id_provincia_indec      13736 non-null  int64  
 1   anio                    13736 non-null  int64  
 2   id_snic                 13736 non-null  object 
 3   nombre_snic             13736 non-null  object 
 4   cantidad_hechos         13736 non-null  int64  
 5   cantidad_victimas       0 non-null      float64
 6   cantidad_victimas_masc  0 non-null      float64
 7   cantidad_victimas_fem   0 non-null      float64
 8   cantidad_victimas_sd    0 non-null      float64
 9   tasa_hechos             13736 non-null  object 
 10  tasa_victimas           0 non-null      object 
 11  tasa_victimas_masc      0 non-null      object 
 12  tasa_victimas_fem       0 non-null      object 
dtypes: float64(4), int64(3), object(6)
memory usage: 1.5+ MB


That's what we have guessed, if `cantidad_victimas` is missing, then it would be possible to calculate the others `victim` columns.

But does this column has missing values? It is because the amount of victimas that can be considered directly affected by the crime are `zero` or is because they cannot be measured?

The [User manual](https://minsegar-my.sharepoint.com/personal/dnec_minseg_gob_ar/Documents/Argentina.gob.ar/Bases/Bases%20SNIC/Manual%20de%20Usuario%20Base%20SNIC/Manual_de_usuario_Base_SNIC_2023.pdf?ga=1) of the original dataset, says in relation to the page 33:

"
Sólo disponible para los tipos de delitos: 1_Homicidios
dolosos, 2_Homicidios dolosos en grado de tentativa,
3_Muertes en Accidentes Viales, 4_Homicidios culposos
por otros hechos, 5_Lesiones dolosas, 6_Lesiones
culposas en Accidentes Viales, 7_Lesiones culposas por
otros hechos, 8_Otros delitos contra las personas,
9_Delitos contra el honor, 10_Violaciones, 11_Otros
delitos contra la integridad sexual, 14_Otros delitos
contra la libertad, 14_1 Trata de personas simple, 14_2
Trata de personas agravado, 14_3 Otros delitos contra la
libertad, 31_Suicidios (consumados).
En los casos en que la jurisdicción envió información
rectificada a nivel total provincial o sin desagregación
por sexo, se asignaron las cantidades a víctimas con
sexo sin determinar.
"

So there is a systematic missing of this values and it covers more than 50% of the dataset rows. They are simply not provided.

Let´s see how much of the total sum of crimes those missing values represent.


In [216]:
non_missing_val_crimes = crimes[~(crimes["cantidad_victimas"].isna())]

In [219]:
missing_val_crimes_sum = missing_val_crimes["cantidad_hechos"].sum()
non_missing_val_crimes_sum = non_missing_val_crimes["cantidad_hechos"].sum()

In [221]:
print(f"There are {missing_val_crimes_sum} registered values with missing values for the victims")
print(f"There are {non_missing_val_crimes_sum} registered values with NON missing values for the victims")

There are 29044815 registered values with missing values for the victims
There are 7160626 registered values with NON missing values for the victims


There are a lot of crimes with missing values for the victims. So drop them doesn´t seems like a good idea. They can tell us a lot about the amount of crime in an specific province.

We will keep them, we just have to be careful with this in the Analysis part.

We just need to change some datatypes that are incorrect.

In [223]:
crimes

,id_provincia_indec,anio,id_snic,nombre_snic,cantidad_hechos,cantidad_victimas,cantidad_victimas_masc,cantidad_victimas_fem,cantidad_victimas_sd,tasa_hechos,tasa_victimas,tasa_victimas_masc,tasa_victimas_fem
0,2,2000,1,Homicidios dolosos,149,158.0,121.0,37.0,0.0,"4,9750013","5,2755051","8,8257008","2,2783601"
1,2,2000,2,Homicidios dolosos en grado de tentativa,54,58.0,40.0,18.0,0.0,"1,8030207","1,9365778","2,9175868","1,1083914"
2,2,2000,3,Muertes en accidentes viales,121,126.0,97.0,29.0,0.0,"4,040102","4,2070484","7,0751481","1,7857418"
3,2,2000,4,Homicidios culposos por otros hechos,35,36.0,20.0,13.0,3.0,"1,1686245","1,2020137","1,4587934",",80050492"
4,2,2000,5,Lesiones dolosas,13154,13732.0,8105.0,5485.0,142.0,"439,20248","458,50146","591,17603","337,7515"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21339,94,2022,29_7,Contrabando de elementos nucleares agresivos q...,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
21340,94,2022,29_8,"Otros delitos previstos en leyes especiales n,c,p",3082,NaN,NaN,NaN,NaN,"1693,5648",NaN,NaN,NaN
21341,94,2022,30,Contravenciones,557,NaN,NaN,NaN,NaN,"306,07254",NaN,NaN,NaN
21342,94,2022,31,Suicidios (consumados),22,22.0,12.0,10.0,0.0,"13,150816","13,150816","14,091123","12,175818"


The amount of `victims` shoul be an `int` and the `tasa` related columns a float. But this one, as in Spanish we use the "," insted of "." for decimals, is read as an string

In [226]:
problematic_cols = ["tasa_hechos", "tasa_victimas", "tasa_victimas_masc", "tasa_victimas_fem"]

In [227]:
for col in problematic_cols:
  crimes[col] =  crimes[col].str.replace("," , ".")


In [229]:
crimes[problematic_cols] = crimes[problematic_cols].astype(float)

Now we have to change to `int` the nominal amount of victims

In [232]:
crimes[["cantidad_victimas", "cantidad_victimas_masc", "cantidad_victimas_fem",	"cantidad_victimas_sd"]] = crimes[["cantidad_victimas", "cantidad_victimas_masc", "cantidad_victimas_fem",	"cantidad_victimas_sd"]].round(0).astype('Int64')

In [234]:
crimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id_provincia_indec      21344 non-null  int64  
 1   anio                    21344 non-null  int64  
 2   id_snic                 21344 non-null  object 
 3   nombre_snic             21344 non-null  object 
 4   cantidad_hechos         21344 non-null  int64  
 5   cantidad_victimas       7608 non-null   Int64  
 6   cantidad_victimas_masc  7608 non-null   Int64  
 7   cantidad_victimas_fem   7608 non-null   Int64  
 8   cantidad_victimas_sd    7608 non-null   Int64  
 9   tasa_hechos             21344 non-null  float64
 10  tasa_victimas           7608 non-null   float64
 11  tasa_victimas_masc      7608 non-null   float64
 12  tasa_victimas_fem       7608 non-null   float64
dtypes: Int64(4), float64(4), int64(3), object(2)
memory usage: 2.2+ MB


In [222]:
del missing_val_crimes, non_missing_val_crimes, missing_val_crimes_sum, non_missing_val_crimes_sum, problematic_cols

### Suicides

In [235]:
suicides

,id_hecho,id_departamento_indec,fecha,hora,lugar,modalidad,origen_registro,suicida_sexo,suicida_rango_edad,suicida_identidad_genero
2,8569,78021,01-01-2017,19:00:00,Vía pública,Ahorcamiento,Denuncia particular,Masculino,15-19,Sin determinar
3,8705,58035,31-01-2017,17:30:00,Otro Lugar ( Especificar ),Arma de fuego,Intervención policial,Masculino,20-24,Sin determinar
4,8706,58035,31-01-2017,22:40:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,20-24,Sin determinar
5,8707,58035,29-01-2017,20:30:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,20-24,Sin determinar
6,8708,58049,02-01-2017,14:25:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,45-49,Sin determinar
...,...,...,...,...,...,...,...,...,...,...
21713,38831,14021,28-08-2022,11:11:11,Domicilio particular,Ahorcamiento,Intervención policial,Femenino,15-19,Mujer
21714,37640,18049,09-07-2022,07:30:00,Cárcel o comisaría,Ahorcamiento,Intervención policial,Masculino,30-34,Varón
21715,37642,18070,11-09-2022,05:00:00,Domicilio particular,Sin determinar,Intervención policial,Masculino,50-54,Varón
21716,37643,18070,27-09-2022,06:45:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,15-19,Varón


In [236]:
suicides.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21426 entries, 2 to 21717
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id_hecho                  21426 non-null  int64 
 1   id_departamento_indec     21426 non-null  int64 
 2   fecha                     21426 non-null  object
 3   hora                      21426 non-null  object
 4   lugar                     21426 non-null  object
 5   modalidad                 21426 non-null  object
 6   origen_registro           21426 non-null  object
 7   suicida_sexo              21426 non-null  object
 8   suicida_rango_edad        21426 non-null  object
 9   suicida_identidad_genero  21426 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.8+ MB


We don´t have any `Null` value. The datatypes look correct.

However, it would be useful to see the values for our categorical variables. It seems like `suicida_identidad_genero` has "Sin Determinar" vals.

In [237]:
categorical_cols = ["fecha", "hora",	"lugar", 	"modalidad", "origen_registro", "suicida_sexo",	"suicida_rango_edad", "suicida_identidad_genero"]

In [240]:
for col in categorical_cols:
  print(suicides[col].value_counts())
  print("------------------------------")

01-01-2021    27
01-01-2019    25
13-12-2021    24
20-01-2019    22
16-10-2022    22
              ..
31-05-2019     2
20-12-2017     2
26-07-2019     1
20-02-2017     1
08-06-2020     1
Name: fecha, Length: 2191, dtype: int64
------------------------------
00:00:00    506
12:00:00    337
11:00:00    315
18:00:00    309
11:11:11    304
           ... 
00:44:00      1
21:41:00      1
17:36:00      1
05:42:00      1
04:46:00      1
Name: hora, Length: 1528, dtype: int64
------------------------------
Domicilio particular          16137
Vía pública                    2294
Sin determinar                 1261
Otro Lugar ( Especificar )     1230
Vías del FF.CC.                 276
Cárcel o comisaría              228
Name: lugar, dtype: int64
------------------------------
Ahorcamiento                        14472
Arma de fuego                        3381
Sin determinar                       1380
Se arroja al vacío                    747
Envenenamiento                        568
Se arroja a l

Only in the `suicida_identidad_genero` the majority of rows have the "Sin determinar" value.

The [User Manual](https://minsegar-my.sharepoint.com/personal/dnec_minseg_gob_ar/Documents/Argentina.gob.ar/Bases/Bases%20SAT/Bases%20SAT%20SS/Manual%20de%20usuario%20de%20la%20base%20SAT%20SS/Manual_de_usuario_Base_SAT-SS_2022.pdf?ga=1) (page 29) says in relation to this:

"
De acuerdo a la Ley de Identidad de Género Nro. 26.743, se entiende por identidad de
género: “la vivencia interna e individual del género tal como cada persona la siente, la
cual puede corresponder o no con el sexo asignado a momento del nacimiento,
incluyendo la vivencia personal del cuerpo. Esto puede involucrar la modificación de la
apariencia o la función corporal a través de medios farmacológicos, quirúrgicos o de otra
índole, siempre que ello sea libremente”

[...]

99.Sin determinar
Se utiliza cuando no se dispone de información suficiente para determinar la
identidad de género de la persona.

"

Based on the fact that this column is related to a personal experience of the suicidial, it makes sense that most of the data will be missing.

Anyways, we will keep all the rows, since the "Sin determinar" value is not predominant for the other columns.

### Mean Income

In [241]:
mean_income

,fecha,id_departamento_indec,clae2,w_mean
0,2014-01-01,2000,1,13085
1,2014-01-01,2000,2,10081
2,2014-01-01,2000,3,17837
3,2014-01-01,2000,5,57688
4,2014-01-01,2000,6,62211
...,...,...,...,...
3597392,2023-09-01,94014,93,536119
3597393,2023-09-01,94014,94,504914
3597394,2023-09-01,94014,95,367788
3597395,2023-09-01,94014,96,316829


In [242]:
mean_income.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3587653 entries, 0 to 3597396
Data columns (total 4 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   fecha                  object
 1   id_departamento_indec  int64 
 2   clae2                  int64 
 3   w_mean                 int64 
dtypes: int64(3), object(1)
memory usage: 136.9+ MB


Nothing to do here. The datatypes are correct. We just have to be careful with the dates, consider the specific format that this dataset uses `yyyy-mm-dd`

### CLAE2

In [243]:
clae2

,clae2,clae2_desc
0,1,"Agricultura, ganadería, caza y servicios relac..."
1,2,Silvicultura y explotación forestal
2,3,Pesca y acuicultura
3,5,Extracción de carbón y lignito
4,6,Extracción de petróleo crudo y gas natural
...,...,...
81,93,"Actividades deportivas, recreativas y de entre..."
82,94,Servicios de asociaciones
83,95,Reparación de computadoras y equipos de uso do...
84,96,Otros servicios personales


In [244]:
clae2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86 entries, 0 to 85
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   clae2       86 non-null     int64 
 1   clae2_desc  86 non-null     object
dtypes: int64(1), object(1)
memory usage: 2.0+ KB


## Saving the datasets

In [245]:
clean_data_dir = current_directory + "Clean_Data/"

In [246]:
provinces.to_csv(clean_data_dir+"provincias.csv")

In [247]:
departments.to_csv(clean_data_dir+"departamentos.csv")

In [248]:
crimes.to_csv(clean_data_dir+"crimenes.csv")

In [249]:
suicides.to_csv(clean_data_dir+"suicidios.csv")

In [250]:
mean_income.to_csv(clean_data_dir+"ingresos_promedio.csv")

In [252]:
clae2.to_csv(clean_data_dir+"clae2.csv")

In [ ]:
del clean_data_dir, province